In [ ]:
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors

import toolpac.calc.binprocessor_v2 as bp

import dataTools.data.BinnedData as bin_tools
from dataTools.data import DataCollection
from dataTools.data import data_getter

### Trials

In [ ]:
N1 = 50
x1 = np.linspace(0,10,N1)
y1 = x1 + 5*np.random.rand(N1)

N2 = 15
x2 = np.linspace(0,10,N2)
y2 = x2 + 5*np.random.rand(N2)


weights = [] # depends on bin_edges



In [ ]:
bin_edges = [0,7.5,15]
bci1d = bp.Bin1D(xbinlimits = bin_edges)

binned1 = bp.BinnedData1D(x1,y1, bci1d)
binned2 = bp.BinnedData1D(x2,y2, bci1d)

fig, ax = plt.subplots()
ax.scatter(x1,y1)
ax.scatter(x2,y2)

for x in bin_edges:
    ax.hlines(x,0,10, ls='dotted', color='lightgrey', zorder=0)

for (mean, ylims) in zip(binned1.vmean, ([0,7.5], [7.5,15])):
    ax.vlines(mean, *ylims, ls='dashed', color='tab:blue')

for (mean, ylims) in zip(binned2.vmean, ([0,7.5], [7.5,15])):
    ax.vlines(mean, *ylims, ls='dashed', color='tab:orange')

In [ ]:
bin_weights1 = [1/i for i in binned1.vcount]
bin_weights2 = [1/i for i in binned2.vcount]
bin_weights1, bin_weights2

weight_per_point = [np.array([1/i]*int(i)) for i in binned1.vcount]
weight_per_point

In [ ]:
for bin_i in [0,1]:
    print(binned1.vbindata[bin_i], sum(binned1.vbindata[bin_i]*weight_per_point[bin_i]), binned1.vmean[bin_i], '\n')

In [ ]:
# For bin i
bin_i = 0

v_A = binned1.vbindata[bin_i]
v_B = binned2.vbindata[bin_i]

N_A = binned1.vcount[bin_i]
N_B = binned2.vcount[bin_i]

(1/N_A * sum(v_A) + 1/N_B * sum(v_B)) / ( N_A/N_A + N_B/N_B)

In [ ]:
(1/N_A * sum(v_A) + 1/N_B * sum(v_B)) / 2

In [ ]:
# How to generalise for more than 2 data arrays

def sample_x_y(N): 
    x = np.linspace(0,10, N)
    y = x + 5*np.random.rand(N)
    return N, x, y

param_list = []
for N in [50,15,30,100]:
    (N,x,y) = sample_x_y(N)
    param_list.append((N,x,y))

In [ ]:
def get_weighted_sum(param_list):
    """ param_list: List of (N,x_arr,y_arr)"""
    total_weight = len(param_list) # nr of ascents

    weighted_sum_list = []
    for (x_arr, y_arr) in param_list:
        N = len(y_arr)
        weighted_sum = sum(y_arr) * 1/N
        weighted_sum_list.append(weighted_sum)

    return sum(weighted_sum_list) / total_weight 

In [ ]:
bin_edges = [0,7.5,15]
bci1d = bp.Bin1D(xbinlimits=bin_edges)

binned1 = bp.BinnedData1D(x1,y1, bci1d)
binned2 = bp.BinnedData1D(x2,y2, bci1d)

fig, ax = plt.subplots()
ax.scatter(x1,y1)
ax.scatter(x2,y2)

for x in bin_edges:
    ax.hlines(x,0,10, ls='dotted', color='lightgrey', zorder=0)

for (mean, ylims) in zip(binned1.vmean, ([0,7.5], [7.5,15])):
    ax.vlines(mean, *ylims, ls='dashed', color='tab:blue')

for (mean, ylims) in zip(binned2.vmean, ([0,7.5], [7.5,15])):
    ax.vlines(mean, *ylims, ls='dashed', color='tab:orange')

### Implementation - random example data

In [ ]:
# Weighted and unweighted binning

# Prepare binning
bin_edges = [0,2.5,5,6.5, 8.5,10]

LIST_N = [20,70,400]
colors = list(mcolors.TABLEAU_COLORS.values())

bci1d = bp.Bin1D(xbinlimits = bin_edges)

# Example data: x-array and y-array
def sample_x_v(N): 
    x = np.linspace(0,10, N)
    v = x**0.8 + 2*np.random.rand(N) + 0.3*N**0.5
    return x, v
data_list = []
for N in LIST_N:
    (x,v) = sample_x_v(N)
    data_list.append((x,v))

# Calculate non-weighted mean for each bin
x_simple = np.concatenate([i[0] for i in data_list])
v_simple = np.concatenate([i[1] for i in data_list])

unweighted_binned = bp.BinnedData1D(v_simple, x_simple, bci1d)

# Sort into bins on x
binned_list = []
for x_v_tuple in data_list:
    x,v = x_v_tuple 
    binned = bp.BinnedData1D(v, x, bci1d)
    binned_list.append(binned)

# Now calculate weighted mean per bin
vmean_per_bin = []
for i in np.arange(bci1d.nx): # Bin index
    weighted_sum_list = []
    for j, binned in enumerate(binned_list): # Ascent index
        v_arr = binned.vbindata[i]
        if str(v_arr)=='nan': continue
        weight = 1/len(v_arr)
        weighted_sum = sum(v_arr) * weight
        weighted_sum_list.append(weighted_sum)
    
    total_weight = len(weighted_sum_list)
    weighted_sum = sum(weighted_sum_list) / total_weight
    vmean_per_bin.append(weighted_sum)


In [ ]:
# Plot the results
fig, ax = plt.subplots(dpi=150)
for x in bin_edges:
    ax.hlines(x,0,20, ls='dotted', color='lightgrey', zorder=0)

for i in np.arange(bci1d.nx):
    # show normal means
    for j, binned in enumerate(binned_list):
        ax.scatter(binned.vmean[i], binned.xintm[i], 
                   color=colors[j], s=60, marker='d', 
                   zorder=10, label = LIST_N[j] if i==0 else '',
                   edgecolor='k', lw=0.7)
        ax.scatter(binned.vbindata[i], binned.xbindata[i], 
                   color=colors[j], s=15, edgecolor='None', alpha=0.6)
    
    # show weighted means
    xlims = bci1d.xint[i], bci1d.xint[i]+bci1d.xbsize[i]
    ax.vlines(vmean_per_bin[i], *xlims, 
              color = 'k', lw=2,
              label ='x$_w$' if i==0 else '')
    
    # show non-weighted mean
    ax.vlines(unweighted_binned.vmean[i], *xlims, 
              color='tab:red',lw=2,
              label ='x' if i==0 else '')

ax.legend(title='Mean / N')

In [ ]:
# Try to calculate the weighted standard deviation - get data from above

# HAVE: binned_list, bci1d

# Now calculate weighted mean per bin
# vmean_per_bin = []
# for i in np.arange(bci1d.nx): # Bin index
    
i = 0

weighted_sum_list = []
total_vbindata = []
weight_list = []

for j, binned in enumerate(binned_list): # Ascent index
    v_arr = binned.vbindata[i]
    if str(v_arr)=='nan': continue
    weight = 1/len(v_arr)
    
    weight_list.append([weight]*len(v_arr))

    weighted_sum = sum(v_arr) * weight 
    weighted_sum_list.append(weighted_sum)
    
    total_vbindata += list(v_arr)

total_weight = len(weighted_sum_list)
weighted_mean = sum(weighted_sum_list) / total_weight

# calculate the weighted standard deviation
diff_squared = [(val-weighted_mean)**2 for val in total_vbindata]
weighted_diffs = [weight*ds for weight, ds in zip(np.concatenate(weight_list), diff_squared)]
weighted_var = (np.nansum(weighted_diffs) / total_weight)
weighted_std = weighted_var**0.5
    
weighted_mean, weighted_std 

In [ ]:
import numpy as np
import toolpac.calc.binprocessor_v2 as bp

# Creating a function for weighted binning
def weighted_binning(data_list, binclassinstance): 
    """ Weighted binning where each separate item in data_list contributes an equal amount (= 1). 
    
    data_list (list[(v,x)]): List containing tuples with (v,x) data arrays
    binclassinstance (bp.Bin_(not)equi1d): one-dimensional binning structure
    
    Returns lists of weighted mean and weighted standard deviation.
    """
    # Sort into bins on x
    binned_list = []
    for x_v_tuple in data_list:
        x,v = x_v_tuple 
        binned = bp.BinnedData1D(v, x, binclassinstance)
        binned_list.append(binned)

    # Now calculate weighted mean and std per bin
    weighted_mean_per_bin = []
    weighted_std_per_bin = []
    for i in np.arange(binclassinstance.nx): # Bin index
        weighted_sum_list = []
        total_vbindata = []
        weight_list = []

        for j, binned in enumerate(binned_list): # Ascent index
            try: v_arr = binned.vbindata[i]
            except: print('vbd', binned.vbindata); continue
            if str(v_arr)=='nan': continue
            weight = 1/len(v_arr)

            weighted_sum_ascent = np.nansum(v_arr) * 1/len(v_arr) 
            weighted_sum_list.append(weighted_sum_ascent)
            
            weight_list.append([weight]*len(v_arr))
            total_vbindata += list(v_arr)

        total_weight = len(weighted_sum_list)
        weighted_mean = sum(weighted_sum_list) / total_weight

        # calculate the weighted standard deviation
        diff_squared = [(val-weighted_mean)**2 for val in total_vbindata]
        weighted_diffs = [weight*ds for weight, ds in zip(np.concatenate(weight_list), diff_squared)]
        weighted_var = (np.nansum(weighted_diffs) / total_weight)
        weighted_std = weighted_var**0.5
        
        weighted_mean_per_bin.append(weighted_mean)
        weighted_std_per_bin.append(weighted_std)

    return weighted_mean_per_bin, weighted_std_per_bin


# Example data
def sample_x_v(N): 
    x = np.linspace(0,10, N)
    v = x + 10*np.random.rand(N)
    return x, v

LIST_N = [2,15,300,20]
data_list = []
for N in LIST_N:
    (x,v) = sample_x_v(N)
    data_list.append((x,v))

# Prepare binning
bin_edges = [0,2.5,5,6.5, 8.5,10]
bci1d = bp.Bin1D(xbinlimits=bin_edges)

mean, std = weighted_binning(data_list, bci1d)
[(m,s) for m,s in zip(mean, std)]

### Weighted binning for climatology creation

In [ ]:
def monthly_climatology(GlobalObject, subs, vcoord, bin_kwargs):
    """ Create monthly binned profiles of subs (give data for a latitude band)
    
    Parameters: 
        GlobalObject (GlobalData)
        subs (dcts.Substance)
        vcoord (dcts.Coordinate): Vertical coord. for profile
        lat_bsize (float): Size of latitude bands (even)
    """    
    bci = bin_tools.make_bci(vcoord, **bin_kwargs)

    binned_monthly = {}

    for month in set(GlobalObject.df.index.month):
        data_month = GlobalObject.sel_month(month)

        data_list = []
        for flight_nr in data_month.flights:
            data_flight = data_month.sel_flight(flight_nr)
            
            x = data_flight.get_var_data(vcoord)
            v = data_flight.get_var_data(subs)
            data_list.append((x,v))

        mean, std = bin_tools.weighted_binning(data_list, bci)
        binned_monthly[month] = (mean, std)

    return binned_monthly

In [ ]:
from importlib import reload; reload(data_getter)
pdir = r"C:\Users\sophie_bauchinger\Documents\GitHub\chemTPanalyses\chemTPanalyses\data\store"
stn_dict, station_df = data_getter.load_ozone_sonde_data(["205", "254"], pdir=pdir)

woudc_data = DataCollection(station_df, 'WOUDC')
woudc_data.data.update(**stn_dict)

# Define subs / coord
[o3_subs] = woudc_data.get_substs(short_name='o3', model='MSMT', unit='ppb')
[pt_coord] = woudc_data.get_coords(vcoord='pt', model='MSMT')

# monthly_climatology(woudc_data, o3_subs, pt_coord, {'xbmax':1000})

In [ ]:
from importlib import reload; reload(bin_tools)
GlobalObject = woudc_data
subs = o3_subs
vcoord = pt_coord

bci = bin_tools.make_bci(vcoord, xbmax=1000)
# bci = bin_tools.make_bci(vcoord)

binned_monthly = {}

for month in set(GlobalObject.df.index.month):
    data_month = GlobalObject.sel_month(month)

    data_list = []
    for flight_nr in data_month.flights:
        data_flight = data_month.sel_flight(flight_nr)
        
        x = data_flight.get_var_data(vcoord)
        v = data_flight.get_var_data(subs)
        data_list.append((x,v))

    mean, std = bin_tools.weighted_binning(data_list, bci)
    binned_monthly[month] = (mean, std)

In [ ]:
import dataTools.dictionaries as dcts

%matplotlib inline
fig, ax = plt.subplots(figsize=(5,8))

for month, (w_mean, std) in binned_monthly.items():
    ax.plot(w_mean, bci.xintm, color = dcts.dict_month()[f"color_{month}"])

In [ ]:
np.nanmin([np.nanmin(i[0]) for i in binned_monthly.values()]),
np.nanmax([np.nanmax(i[0]) for i in binned_monthly.values()])

In [ ]:
bci.xintm[0]-50*bci.xbsize

In [ ]:
import dataTools.plot.create_figure as cfig; reload(cfig)
fig, axs = plt.subplots(3,4, sharex=True, sharey=True, figsize=(8,8))

axs.flat[0].set_ylim(bci.xintm[0]-bci.xbsize, bci.xintm[-1]+bci.xbsize)
axs.flat[0].set_xlim(
    np.nanmin([np.nanmin(i[0]) for i in binned_monthly.values()]) - 400,
    np.nanmax([np.nanmax(i[0]) for i in binned_monthly.values()]))
for (month, (w_mean, std)), ax in zip(binned_monthly.items(), axs.flat):
    ax.scatter(
        GlobalObject.sel_month(month).get_var_data(subs),
        GlobalObject.sel_month(month).get_var_data(vcoord),
        alpha = 0.2, edgecolor="None", color = dcts.dict_month()[f"color_{month}"])
    ax.plot(w_mean, bci.xintm, color = 'k', path_effects = [cfig.outline(lw=2)], lw=2)

for ax in axs[-1,:]:
    ax.set_xlabel(subs.label())
for ax in axs[:,0]: 
    ax.set_ylabel(vcoord.label())
fig.tight_layout()
fig.subplots_adjust(top=0.95)
fig.suptitle("WOUDC climatology for normalised ozone sonde msmts at stations 205 and 245")

In [ ]:
import dataTools.plot.create_figure as cfig; reload(cfig)
fig, axs = plt.subplots(3,4, sharex=True, sharey=True, figsize=(8,8))

axs.flat[0].set_ylim(bci.xintm[0]-bci.xbsize, 600)
axs.flat[0].set_xlim(
    np.nanmin([np.nanmin(i[0]) for i in binned_monthly.values()])-500, 7000)
for (month, (w_mean, std)), ax in zip(binned_monthly.items(), axs.flat):
    ax.scatter(
        GlobalObject.sel_month(month).get_var_data(subs),
        GlobalObject.sel_month(month).get_var_data(vcoord),
        alpha = 0.2, edgecolor="None", color = dcts.dict_month()[f"color_{month}"])
    ax.scatter(w_mean, bci.xintm, color = 'k', path_effects = [cfig.outline(lw=2)], lw=2)

for ax in axs[-1,:]:
    ax.set_xlabel(subs.label())
for ax in axs[:,0]: 
    ax.set_ylabel(vcoord.label())
fig.tight_layout()
fig.subplots_adjust(top=0.95)
fig.suptitle("WOUDC climatology for normalised ozone sonde msmts at stations 205 and 245")

### Weighted binning class (1D and 2D)

In [ ]:
def monthly_weighted_clim(GlobalObject, subs, vcoord, bin_kwargs):
    """ Create monthly binned profiles of subs (give data for a latitude band)
    
    Parameters: 
        GlobalObject (GlobalData)
        subs (dcts.Substance)
        vcoord (dcts.Coordinate): Vertical coord. for profile
        lat_bsize (float): Size of latitude bands (even)
    """    
    bci = bin_tools.make_bci(vcoord, **bin_kwargs)

    binned_monthly = {}

    for month in set(GlobalObject.df.index.month):
        data_month = GlobalObject.sel_month(month)

        data_list = []
        for flight_nr in data_month.flights:
            data_flight = data_month.sel_flight(flight_nr)
            
            x = data_flight.get_var_data(vcoord)
            v = data_flight.get_var_data(subs)
            data_list.append((v,x))

        binned = bp.WeightedBinning1D(data_list, bci)
        mean, std = binned.weighted_mean, binned.weighted_std
        binned_monthly[month] = (mean, std)

    return binned_monthly

In [ ]:
from importlib import reload; reload(data_getter)
pdir = r"C:\Users\sophie_bauchinger\Documents\GitHub\chemTPanalyses\chemTPanalyses\data\store"
stn_dict, station_df,_ = data_getter.load_ozone_sonde_data(["205", "254"], pdir=pdir)

woudc_data = DataCollection(station_df, 'WOUDC')
woudc_data.data.update(**stn_dict)

# Define subs / coord
[o3_subs] = woudc_data.get_substs(short_name='o3', model='MSMT', unit='ppb')
[pt_coord] = woudc_data.get_coords(vcoord='pt', model='MSMT')

In [ ]:
%matplotlib inline
import dataTools.dictionaries as dcts

binned_monthly = monthly_weighted_clim(
    woudc_data, o3_subs, pt_coord, {'xbmax':1000})
bci = bin_tools.make_bci(pt_coord, xbmax=1000)

fig, ax = plt.subplots(figsize=(5,8))
for month, (w_mean, std) in binned_monthly.items():
    ax.plot(w_mean, bci.xintm, color = dcts.dict_month()[f"color_{month}"])

In [ ]:
import dataTools.plot.create_figure as cfig; reload(cfig)
fig, axs = plt.subplots(3,4, sharex=True, sharey=True, figsize=(8,8))

axs.flat[0].set_ylim(bci.xintm[0]-bci.xbsize, bci.xintm[-1]+bci.xbsize)
axs.flat[0].set_xlim(
    np.nanmin([np.nanmin(i[0]) for i in binned_monthly.values()]) - 400,
    np.nanmax([np.nanmax(i[0]) for i in binned_monthly.values()]))
for (month, (w_mean, std)), ax in zip(binned_monthly.items(), axs.flat):
    ax.scatter(
        woudc_data.sel_month(month).get_var_data(o3_subs),
        woudc_data.sel_month(month).get_var_data(pt_coord),
        alpha = 0.2, edgecolor="None", color = dcts.dict_month()[f"color_{month}"])
    ax.plot(w_mean, bci.xintm, color = 'k', path_effects = [cfig.outline(lw=2)], lw=2)

for ax in axs[-1,:]:
    ax.set_xlabel(o3_subs.label())
for ax in axs[:,0]: 
    ax.set_ylabel(pt_coord.label())
fig.tight_layout()
fig.subplots_adjust(top=0.95)
fig.suptitle("WOUDC climatology for normalised ozone sonde msmts at stations 205 and 245")